# OpenBnB MCP Sunucusu Entegrasyonu ile Semantic Kernel

Bu not defteri, MCPStdioPlugin kullanarak gerçek Airbnb konaklamalarını aramak için OpenBnB MCP sunucusuyla Semantic Kernel'in nasıl kullanılacağını gösterir. LLM erişimi için Azure AI Foundry kullanılır. Ortam değişkenlerinizi ayarlamak için [Kurulum Dersi](/00-course-setup/README.md) bağlantısını takip edebilirsiniz.


## Gerekli Paketleri İçe Aktarın


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP Eklenti Bağlantısını Oluşturma

MCPStdioPlugin kullanarak [OpenBnB MCP sunucusuna](https://github.com/openbnb-org/mcp-server-airbnb) bağlanacağız. Bu sunucu, @openbnb/mcp-server-airbnb paketi aracılığıyla Airbnb arama işlevselliği sağlar.


## İstemciyi Oluşturma

Bu örnekte, LLM erişimimiz için Azure AI Foundry kullanacağız. Çevre değişkenlerinizin doğru bir şekilde ayarlandığından emin olun.


## Ortam Yapılandırması

Azure OpenAI ayarlarını yapılandırın. Aşağıdaki ortam değişkenlerinin ayarlandığından emin olun:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP Entegrasyonunu Anlama

Bu not defteri, gerçek Airbnb arama işlevselliği sağlayan **gerçek OpenBnB MCP sunucusuna** bağlanır.

### Nasıl çalışır:

1. **MCPStdioPlugin**: MCP sunucusuyla standart giriş/çıkış iletişimi kullanır
2. **Gerçek NPM Paketi**: `@openbnb/mcp-server-airbnb` paketini npx aracılığıyla indirir ve çalıştırır
3. **Canlı Veri**: Airbnb API'lerinden gerçek mülk verilerini döndürür
4. **Fonksiyon Keşfi**: Aracı, MCP sunucusundan mevcut fonksiyonları otomatik olarak keşfeder

### Mevcut Fonksiyonlar:

OpenBnB MCP sunucusu genellikle aşağıdaki fonksiyonları sağlar:
- **search_listings** - Konum ve kriterlere göre Airbnb mülklerini arayın
- **get_listing_details** - Belirli mülkler hakkında ayrıntılı bilgi alın
- **check_availability** - Belirli tarihler için müsaitlik kontrolü yapın
- **get_reviews** - Mülkler için yorumları alın
- **get_host_info** - Mülk sahipleri hakkında bilgi alın

### Gereksinimler:

- Sisteminizde kurulu **Node.js**
- MCP sunucu paketini indirmek için **internet bağlantısı**
- **NPX** (Node.js ile birlikte gelir)

### Bağlantıyı Test Etme:

MCP sunucusunu manuel olarak çalıştırarak test edebilirsiniz:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Bu işlem, OpenBnB MCP sunucusunu indirip başlatır ve ardından Semantic Kernel gerçek Airbnb verilerine bağlanır.


## OpenBnB MCP Sunucusu ile Ajanı Çalıştırma

Şimdi, OpenBnB MCP sunucusuna bağlanan ve Stockholm'de 2 yetişkin ve 1 çocuk için gerçek Airbnb konaklamalarını arayan AI Ajanını çalıştıracağız. Arama kriterlerini değiştirmek için `user_inputs` listesini istediğiniz gibi düzenleyebilirsiniz.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Özet
Tebrikler! Model Context Protocol (MCP) kullanarak gerçek dünya konaklama aramasıyla entegre olan bir yapay zeka ajanı başarıyla oluşturdunuz:

## Kullanılan Teknolojiler:
- Semantic Kernel - Azure OpenAI ile akıllı ajanlar oluşturmak için
- Azure AI Foundry - LLM yetenekleri ve sohbet tamamlama için
- MCP (Model Context Protocol) - Standart araç entegrasyonu için
- OpenBnB MCP Sunucusu - Gerçek Airbnb arama işlevselliği için
- Node.js/NPX - Harici MCP sunucusunu çalıştırmak için

## Öğrendikleriniz:
- MCP Entegrasyonu: Semantic Kernel ajanlarını harici MCP sunucularına bağlama
- Gerçek Zamanlı Veri Erişimi: Canlı API'ler aracılığıyla gerçek Airbnb mülklerini arama
- Protokol İletişimi: Ajan ve MCP sunucusu arasında stdio iletişimini kullanma
- Fonksiyon Keşfi: MCP sunucularından mevcut fonksiyonları otomatik olarak keşfetme
- Akış Yanıtları: Fonksiyon çağrılarını gerçek zamanlı olarak yakalama ve kaydetme
- HTML İşleme: Ajan yanıtlarını stillendirilmiş tablolar ve etkileşimli görsellerle biçimlendirme

## Sonraki Adımlar:
- Ek MCP sunucularını entegre edin (hava durumu, uçuşlar, restoranlar)
- MCP ve A2A protokollerini birleştiren çoklu ajan sistemi oluşturun
- Kendi veri kaynaklarınız için özel MCP sunucuları oluşturun
- Oturumlar arasında kalıcı sohbet hafızası uygulayın
- Ajanı MCP sunucusu orkestrasyonu ile Azure Functions'a dağıtın
- Kullanıcı kimlik doğrulama ve rezervasyon yetenekleri ekleyin



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
